# Imports

In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dataframes

In [159]:
PATH_EVENTS = "data/events.csv"
PATH_POINTS = "data/points.csv"
PATH_RALLIES = "data/rallies.csv"
PATH_SERVES = "data/serves.csv"

df_events = pd.read_csv(PATH_EVENTS)
df_points = pd.read_csv(PATH_POINTS)
df_rallies = pd.read_csv(PATH_RALLIES)
df_serves = pd.read_csv(PATH_SERVES)

df_events.head(3)

,Unnamed: 0,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time
0,0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00
1,1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92
2,2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92


# Data sanitisation

No need :)

# Questions:
- How does the velocity of the ball change per game/set per player? 
- What is the distribution like?

## Calculating Velocity

### Create new column for elapsed time during the single rally (time it took for ball to go to receiver from hitter)
- It subtracts the time from the previous row

In [160]:
df_events["elapsed_time"] = df_events["time"].diff()

### Calculate single velocity instance by dividing change in position (Euclidian distance) by change in time.

In [192]:
df_events["velocity (yard/second) "] = np.sqrt(np.square(df_events["hitter_x"] - df_events["receiver_x"]) + np.square(df_events["hitter_y"] - df_events["receiver_y"])) / df_events["elapsed_time"]
df_events["elapsed_time"].fillna(0, inplace=True)
df_events.head(3)

,Unnamed: 0,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time,elapsed_time,velocity (yard/second)
0,0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00,0.00,inf
1,1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92,0.92,27.427616
2,2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92,2.00,12.174392


In [162]:
df_events.head(3)

,Unnamed: 0,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,hitter_x,hitter_y,receiver_x,receiver_y,time,elapsed_time,velocity (yard/second)
0,0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,6.50,-0.24,1.03,27.44,0.00,NaN,NaN
1,1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.05,25.59,6.17,1.11,0.92,0.92,27.427616
2,2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,1.42,2.33,4.75,26.45,2.92,2.00,12.174392


### Dropping some columns

In [193]:
df_events_velocity = df_events.drop(["hitter_x","hitter_y","receiver_x","receiver_y"], axis=1)
df_events_velocity.head(3)

,Unnamed: 0,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,time,elapsed_time,velocity (yard/second)
0,0,1,70877,1,Djokovic,Nadal,True,first,serve,forehand,0.00,0.00,inf
1,1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.92,0.92,27.427616
2,2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,2.92,2.00,12.174392


**Note: We can ignore the velocities of services because we instantiate the initial velocity is 0. From this data, we have no way of calculating the velocity of a service**

In [194]:
ARG_VELOCITY_NO_SERVE = df_events_velocity["isserve"] == False
df_events_velocity_rally = df_events_velocity[ARG_VELOCITY_NO_SERVE]
df_events_velocity_rally.head(3)

,Unnamed: 0,rallyid,frameid,strokeid,hitter,receiver,isserve,serve,type,stroke,time,elapsed_time,velocity (yard/second)
1,1,1,70900,2,Nadal,Djokovic,False,first,slice,backhand,0.92,0.92,27.427616
2,2,1,70950,3,Djokovic,Nadal,False,first,topspin,forehand,2.92,2.00,12.174392
5,5,3,71820,2,Nadal,Djokovic,False,second,slice,forehand,37.72,0.96,25.806120


In [190]:
df_points.head(3)

,Unnamed: 0,rallyid,server,returner,winner,reason,serve,strokes,totaltime,x,y,score
0,0,1,Djokovic,Nadal,Djokovic,winner,first,3,0.92,1.92,21.96,"0:0, 15:0"
1,2,3,Djokovic,Nadal,Djokovic,out,second,4,4.16,3.33,-0.39,"0:0, 30:0"
2,4,5,Djokovic,Nadal,Djokovic,ace,second,2,0.40,1.62,17.18,"0:0, 40:0"


In [245]:
df_points.tail(3)

,Unnamed: 0,rallyid,server,returner,winner,reason,serve,strokes,totaltime,x,y,score
139,201,202,Nadal,Djokovic,Djokovic,winner,first,4,3.08,1.88,3.36,"6:3 6:2 5:3, 40:15"
140,203,204,Nadal,Djokovic,Nadal,out,first,12,0.84,8.45,-0.56,"6:3 6:2 5:3, 40:30"
141,205,206,Nadal,Djokovic,Djokovic,out,second,5,4.60,0.59,23.86,"6:3 6:2 6:3, 0:0"


## Adding new columns for game number and set number

In [255]:
def get_set(item):
    """ 
        Returns the set number
    """
    
#     for i in range(len(item)):
#         if item[i] == ",":
#             if len(item[:i]) == 3:
#                 return 1
#             elif len(item[:i]) == 7:
#                 return 2
#             elif len(item[:i]) == 11:
#                 return 3

    game_item = item["score"].split(",")[0]

    # get set
    if game_item.count(":") == 1:
        return 1
    elif game_item.count(":") == 2:
        return 2
    else:
        return 3
            

def get_game(item):
    """
        Returns the game number
    """
    
    game_item = item["score"].split(",")[0]

    first_game, second_game = game_item[-3:].split(":")
    return (int(first_game) + int(second_game) + 1)

# using .apply to pass "set" and "game" as an iterable for all elements of "score" series.
df_points["set"] = df_points.apply(get_set)
df_points["game"] = df_points.apply(get_game)


KeyError: ('score', 'occurred at index Unnamed: 0')

In [165]:
# Merge df's so I can get games and sets for rallyid's
# seperate between sets and games
# plot average velocity of ball per game for both players
# plot average velocity of ball per set for both players
# See difference and distribution

,Unnamed: 0,rallyid,server,returner,winner,reason,serve,strokes,totaltime,x,y
0,0,1,Djokovic,Nadal,Djokovic,winner,first,3,0.92,1.92,21.96
1,1,2,Djokovic,Nadal,__undefined__,second_serve,first,1,0.00,7.42,12.10
2,2,3,Djokovic,Nadal,Djokovic,out,second,4,4.16,3.33,-0.39
3,3,4,Djokovic,Nadal,__undefined__,second_serve,first,1,0.00,4.64,17.69
4,4,5,Djokovic,Nadal,Djokovic,ace,second,2,0.40,1.62,17.18


In [166]:
df_serves.head(2)

,Unnamed: 0,rallyid,server,x,y
0,0,1,Djokovic,1.86,16.80
1,1,3,Djokovic,7.05,16.97
